# DSAA 5002 - Data Mining and Knowledge Discovery in Data Science
---

# Task 1 (50 marks) Data Preprocessing and Analysis

**Background: 
Assuming you are a sentiment analyst at a securities firm, your task is to assess the impact of each news article on the A-share listed companies explicitly mentioned.**

# Q2. Data Analysis - Text Knowledge Mining

# part1- Training Sample Annotation

---
## 1. News Characteristic Observation Experiment

In [ ]:
import pandas as pd

# Read the original Excel file
input_file = 'News_output\\News_training_domain_set_withCompany.xlsx'
df_training_domain_company = pd.read_excel(input_file)

# Create two new DataFrames, one to store rows with single companies and another for rows with multiple companies
df_SingleCompany = df_training_domain_company[df_training_domain_company['Explicit_Company'].str.find(',') == -1]
df_MultiCompany = df_training_domain_company[df_training_domain_company['Explicit_Company'].str.find(',') != -1]

# Save these two new DataFrames into different Excel files
output_file_SingleCompany = 'News_output\\News_training_domain_set_withSingleCompany.xlsx'
output_file_MultiCompany = 'News_output\\News_training_domain_set_withMultiCompany.xlsx'

df_SingleCompany.to_excel(output_file_SingleCompany, index=False)
df_MultiCompany.to_excel(output_file_MultiCompany, index=False)

# Count the number of news items with a single company entity
news_num_in_training_domain_with_SingleCompany = df_SingleCompany.shape[0]
# Count the number of news items with multiple company entities
news_num_in_training_domain_with_MultiCompany = df_MultiCompany.shape[0]

In [ ]:
print(news_num_in_training_domain_with_SingleCompany)
print(news_num_in_training_domain_with_MultiCompany)
print(news_num_in_training_domain_with_SingleCompany/news_num_in_training_domain_with_MultiCompany)

### 2. Training Sample Selection

In [ ]:
# Sample the dataset based on the ratio of single company news to multi-company news
# Determine the total sample count for the training set
total_samples = 100000  # Ajust this number as needed

# Calculate the row ratios for the two DataFrames
num_samples_SingleCompany = int(total_samples * len(df_SingleCompany) / len(df_training_domain_company))
num_samples_MultiCompany = total_samples - num_samples_SingleCompany

print(num_samples_SingleCompany)
print(num_samples_MultiCompany)


In [ ]:
# Extract the respective number of samples from each DataFrame
sample_df_SingleCompany = df_SingleCompany.sample(n=num_samples_SingleCompany, random_state=42)
sample_df_MultiCompany = df_MultiCompany.sample(n=num_samples_MultiCompany, random_state=42)

# Concatenate the sampled DataFrames into one training set
training_set = pd.concat([sample_df_SingleCompany, sample_df_MultiCompany])

# Shuffle the order of the training set
training_set = training_set.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Data restructuring for training: Removing data source
# Remove the NewsSource column
training_set.drop(columns=['NewsSource'], inplace=True)

# Concatenate the contents of Title and NewsContent columns into one column
training_set['News'] = training_set['Title'] + ' ' + training_set['NewsContent']

# Delete the Title and NewsContent columns
training_set.drop(columns=['Title', 'NewsContent'], inplace=True)

# Save the training_set to a file
output_training_file = 'Training_dataset\\Training_set_comprihensive.xlsx'
training_set.to_excel(output_training_file, index=False)

In [ ]:
training_set

## 3. Training Sample Annotation
### 3.1. Stable Guarantee - Split the whole training set

In [3]:
import pandas as pd
training_file = 'Training_dataset\\Unlabeled_Training_set_comprihensive.xlsx'
training_set = pd.read_excel(training_file)

In [4]:
training_set

,NewsID,Explicit_Company,News
0,908514,"天健集团, 西南证券",西南证券给予天健集团买入评级 各业务板块稳步发展 营收逆势提升 西南证券08月23日发布研报...
1,813403,"汇纳科技, 亚太药业, 国睿科技, 楚江新材, 宸展光电, 华阳集团, 金禾实业, 利亚德,...",公司互动丨这些公司披露在3D打印、医药等领域最新情况 7月17日，多家上市公司通过互动平台、...
2,706089,粤水电,粤水电：子公司为邵阳市洞口县分散式风电项目排名第二的中标候选人单位 粤水电（SZ 00206...
3,774347,"美年健康, 明德生物",明德生物：公司暂无设立体检机构计划 每经AI快讯，有投资者在投资者互动平台提问：董秘，您好，...
4,317166,"荃银高科, 福瑞股份, 广弘控股",市场结构分化 中小盘股继续活跃 周四，A股各大指数普遍高开之后，走出明显的分化形势...
...,...,...,...
99995,775099,金沃股份,金沃股份：第二季度“金沃转债”转股290股 金沃股份（SZ 300984，收盘价：25.92...
99996,119329,京东方,简报数据显示本制糖期我国食糖产量同比增长24% 中国糖业协会昨天发布的简报显示，2007...
99997,947425,"中迪投资, 新大正, 云煤能源, 广汇物流, 世联行, 三湘印象, 四川路桥, 我乐家居, ...",午间涨跌停股分析：41只涨停股，1只跌停股，我乐家居6连板，中迪投资7天5板 9月4日午间，...
99998,678753,西南证券,西南证券给予王府井持有评级 有税业态稳步复苏 免税增量逐步落地 西南证券04月29日发布研报...


In [27]:
# Calculate the size of each portion
chunk_size = 5000

# Split the training set into multiple portions
split_training_sets = []
for i in range(0, len(training_set), chunk_size):
    chunk = training_set[i:i + chunk_size]
    split_training_sets.append(chunk)

# split_training_sets now contains each portion after splitting
for i, chunk in enumerate(split_training_sets):
    chunk.to_excel(f'Training_dataset\\Unlabeled_Training_set_part_{i + 1}.xlsx', index=False)

### 3.2 Labled by ERNIE-Bot-turbo(LLM chatBot)

In [1]:
import requests
import json
import re
from queue import Queue

KEY_list = [["twnNehaDitFkkhyFteaYWyDh", "1GrVi74Q9c4Py6oMRB0stjSHszrOuZbu"],
            ["BZ71tU3PgRXW2nNLanvroSXb", "ON1mnkbky9cRA3Rwipismfq5gVlY7IzP"],   
            ["h59D6sXFYrnbRn9SSN4lr5EI", "4AIR7HSWDho8ALYnGgPWLW4B6Lkz84yx"],
            ["Q7s4yWzKpo75iXxTq4cmIA3v", "GARxTGLONmhFPL94YRuUdBAIGzLTex1w"]]  # set BAIDU KEY

# Set up the queue for the key pool
def set_key(keylist: list) -> None:
    qKey.queue.clear()
    for key in keylist:
        qKey.put(key)
 
 
# Drop the current key
def drop_key(key):
    for _ in range(qKey.qsize()):
        t = qKey.get()
        if t != key:
            qKey.put(t)


In [8]:
qKey=Queue()

In [64]:
import time as t
def LLM_Label_result(news):
    if qKey.empty():
        # print("The key queue length is empty, resetting the queue")
        set_key(KEY_list)
    tkey = qKey.get() # Retrieve a key
    
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant?access_token=" + get_access_token(tkey)
    
    prompt_question = "Is the following news related to A-share listed companies positive or negative? For positive, please reply 'Positive news', for negative, please reply 'Negative news', no need to state reasons: "
    prompt_target = news
    prompt = prompt_question + prompt_target
    payload = json.dumps({
        "messages": [
            {
                "role": "user",
                "content": prompt     
            }
        ]
    })
    headers = {
        'Content-Type': 'application/json'
    }
    
    requests.adapters.DEFAULT_RETRIES = 10
    
    while True:
        try:
            response = requests.request("POST", url, headers=headers, data=payload)
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            t.sleep(10)
            print("Was a nice sleep, now let me continue...")
            continue

    qKey.put(tkey) # Put the key back into the queue
#     print(response.text)
    if "Positive news" in response.text:
        return 1
    elif "Negative news" in response.text:
        return 0
    else:
        return response.text
    

def get_access_token(key):
    """
    Generate authentication signature (Access Token) using AK, SK
    :return: access_token or None (if error)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": key[0], "client_secret": key[1]}
    return str(requests.post(url, params=params).json().get("access_token"))

print(LLM_Label_result("　　在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重股股价高位运行的风险，目前主力资金主要针对价值低估的二线展开了挖掘，使得近期上演二线蓝筹轮番上涨行情，建议投资者对该品种重点关注，如（600270）外运发展，该股拥有显著的盈利能力，除权后缺口对股价具有较强的牵引作用，目前涨幅滞后，后市可重点关注。    　　外运龙头国际市场高占有率  　　公司核心业务包括航空货运代理、国际快件及综合物流，在全国拥有35家分公司，80个市级物流中心及近200个物流网点，运营网络辐射全国，同时，在国际上与50多家海外货运代理公司有着通力合作，目前在国际航空快件领域和国际航空货运代理领域分别拥有约40%和10%的市场占有率，公司力争发展成为既有航空运力保障，又有地面网络通道的国内一流的快速物流供应商。该公司主要利润来源是与DHL(德国敦豪)合资的中外运敦豪(双方各持50%)，中外运敦豪的资产规模为13亿元，06年上半年实现净利润32217万元(05年实现净利润42634万元、04年净利润39514万元)，中外运敦豪是DHL全球服务网络的重要组成部分，在中国主要城市设立了56家分公司(其中04年新增6家分公司)。根据DHL的规划，未来3年合资公司的利润复合增长率有望达到14%以上，有能力保证未来业绩增长。  　　货运航空将成新的利润增长点  　　公司参与四川航空集团公司整体改制重组，公司将占新设的四川航空集团有限责任公司注册资本的49%。参股新公司后，可为公司的航空货运代理业务以及航空快递业务提供运力支持，从而大大提高公司的竞争优势，此外06年9月，公司与大韩航空、韩亚投资和新韩投资在京签署了合资组建货运航空公司的正式协议，新组建公司注册资本为6500万美元，外运发展占股权的51%。合资公司将以全货机运营，主要经营国际定期与非定期航空货物运输服务，同时兼营自用飞行器租赁和维修服务、包机业务、航空公司间代理服务、进出口贸易服务及相关地面服务等，货运航空业务有望成为公司未来的业绩增长点。  　　资产增值概念后市强势上行  　　二级市场上，该股三季度业绩平稳增长每股受益高达0.42元，属于典型的高收益高分红的价值型蓝筹股，该股同时还具有资产增值概念，其购买的中国国航目前为公司带来亿元以上的帐面收益，技术上该股除权后留下巨大缺口对股价有一定的牵引作用，作为业绩优秀的二线蓝筹，后市有强烈补涨要求。"))

{"error_code":17,"error_msg":"Open api daily request limit reached"}
{"error_code":17,"error_msg":"Open api daily request limit reached"}


In [31]:
import concurrent.futures
import pandas as pd

# Process the data using multithreading
def process_news(index, row):
    news_combine_content = row['News']
    cleaned_text = re.sub(r'\u3000', '', news_combine_content)
    sentiment = LLM_Label_result(cleaned_text)
    
    time = 0
    while sentiment != 1 and sentiment != 0 and time < 20:
        time+=1
        sentiment = LLM_Label_result(cleaned_text)
    if(index%100 == 0):
        print(f"--Index: {index}, Label: {sentiment}--")
    return index, sentiment


In [32]:
# Calculate the size of each part
for i in range(15, 20):
    print(f"----Labeling Training_set_part_{i + 1}.xlsx----")
    chunk = pd.read_excel(f'Training_dataset\\Training_set_labeled_with_ERNIE-Bot-turbo\\Training_set_part_{i + 1}.xlsx')
    
    # Create a new column "sentiment" to store sentiment labels
    result_df = chunk
    default_value = -1  # Dfault value as -1
    result_df['Sentiment'] = default_value
    sentiment_df = result_df
    # Set up the thread pool
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        results = list(executor.map(lambda x: process_news(*x), sentiment_df.iterrows()))
    # Write the results into the DataFrame
    for index, sentiment in results:
        sentiment_df.at[index, 'Sentiment'] = sentiment
    sentiment_df.to_excel(f'Training_dataset\\Training_set_labeled_with_ERNIE-Bot-turbo\\Training_set_part_{i+1}.xlsx', index=False)
    print(f"------Training_set_part_{i+1}_saved------")

----Training_set_part_16.xlsx is labeling----
--Index: 0,Lable: 1--
--Index: 100,Lable: 1--
--Index: 200,Lable: 1--
--Index: 300,Lable: 1--
--Index: 400,Lable: 1--
--Index: 500,Lable: 1--
--Index: 600,Lable: 1--
--Index: 700,Lable: 1--
--Index: 800,Lable: 1--
--Index: 900,Lable: 1--
--Index: 1000,Lable: 1--
--Index: 1100,Lable: 1--
--Index: 1200,Lable: 0--
--Index: 1300,Lable: 1--
--Index: 1400,Lable: 1--
--Index: 1500,Lable: 1--
--Index: 1600,Lable: 0--
--Index: 1700,Lable: 0--
--Index: 1800,Lable: 1--
--Index: 1900,Lable: 1--
--Index: 2000,Lable: 0--
--Index: 2100,Lable: 1--
--Index: 2200,Lable: 1--
--Index: 2300,Lable: 1--
--Index: 2400,Lable: 1--
--Index: 2500,Lable: 1--
--Index: 2600,Lable: 1--
--Index: 2700,Lable: 1--
--Index: 2800,Lable: 1--
--Index: 2900,Lable: 1--
--Index: 3000,Lable: 1--
--Index: 3100,Lable: 1--
--Index: 3200,Lable: 1--
--Index: 3300,Lable: 0--
--Index: 3400,Lable: 1--
--Index: 3500,Lable: 1--
--Index: 3600,Lable: 0--
--Index: 3700,Lable: 0--
--Index: 3800,La

In [35]:
# Calculate the size of each part
full_list = []
for i in range(0, 20):
    print(f"----Training_set_part_{i + 1}.xlsx is concatenating----")
    chunk = pd.read_excel(f'Training_dataset\\Training_set_labeled_with_ERNIE-Bot-turbo\\Training_set_part_{i + 1}.xlsx')
    full_list.append(chunk)
full_df = pd.concat(full_list, axis=0)
full_df.to_excel(f'Training_dataset\\Training_set_labeled_with_ERNIE-Bot-turbo\\Training_set_labeled_with_ERNIE-Bot-turbo.xlsx', index=False)


----Training_set_part_1.xlsx is concatting----
----Training_set_part_2.xlsx is concatting----
----Training_set_part_3.xlsx is concatting----
----Training_set_part_4.xlsx is concatting----
----Training_set_part_5.xlsx is concatting----
----Training_set_part_6.xlsx is concatting----
----Training_set_part_7.xlsx is concatting----
----Training_set_part_8.xlsx is concatting----
----Training_set_part_9.xlsx is concatting----
----Training_set_part_10.xlsx is concatting----
----Training_set_part_11.xlsx is concatting----
----Training_set_part_12.xlsx is concatting----
----Training_set_part_13.xlsx is concatting----
----Training_set_part_14.xlsx is concatting----
----Training_set_part_15.xlsx is concatting----
----Training_set_part_16.xlsx is concatting----
----Training_set_part_17.xlsx is concatting----
----Training_set_part_18.xlsx is concatting----
----Training_set_part_19.xlsx is concatting----
----Training_set_part_20.xlsx is concatting----


### 3.3. Labeled by Senta(ELM)

In [2]:
import requests
import json
import time as t
import pandas as pd
import re

API_KEY = "CR02Q1OOhWO2Do7eOCTteoW3"
SECRET_KEY = "Pa100yMUiRwdtijGOtF7fDd38HsPw3I5"

def Senta_Label_result(news):
    
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=&access_token=" + get_access_token()
    
    payload = json.dumps({
        "text": news
    })
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    requests.adapters.DEFAULT_RETRIES = 10
    
    while True:
        try:
            response = requests.request("POST", url, headers=headers, data=payload)
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            t.sleep(10)
            print("Was a nice sleep, now let me continue...")
            continue
   
        
    # 解析JSON响应  
    response_json = json.loads(response.text)  

    return response_json

def get_access_token():
    """
    Generate an access token using the AK and SK (Access Key and Secret Key)
    :return: access_token, or None if there's an error
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

print(Senta_Label_result("药明康德：业绩说明会定于8月23日举行 药明康德（SH 603259，收盘价：78.6元）8月11日晚间发布公告称，公司2023年半年度业绩说明会定于2023年8月23日（星期三）上午10:30-11:30，以上证路演中心视频直播和网络互动的形式进行。出席本次业绩说明会的人员有：公司副董事长、全球首席投资官胡正国，执行董事、联席首席执行官陈民章，执行董事、联席首席执行官杨青，首席财务官施明，董事会秘书张远舟，独立董事张新教授。　　2023年1至6月份，药明康德的营业收入构成为：化学业务(WuXiChemistry)占比71.36%，测试业务(WuXiTesting)占比16.38%，生物学业务(WuXiBiology)占比6.53%，细胞及基因疗法CTDMO业务(WuXiATU)占比3.78%，国内新药研发服务部(WuXiDDSU)占比1.81%。　　药明康德的总裁、董事长均是Ge Li(李革)，男，56岁，学历背景为博士。　　截至发稿，药明康德市值为2333亿元。（文章来源：每日经济新闻）"))

{'text': '药明康德：业绩说明会定于8月23日举行 药明康德（SH 603259，收盘价：78.6元）8月11日晚间发布公告称，公司2023年半年度业绩说明会定于2023年8月23日（星期三）上午10:30-11:30，以上证路演中心视频直播和网络互动的形式进行。出席本次业绩说明会的人员有：公司副董事长、全球首席投资官胡正国，执行董事、联席首席执行官陈民章，执行董事、联席首席执行官杨青，首席财务官施明，董事会秘书张远舟，独立董事张新教授。\u3000\u30002023年1至6月份，药明康德的营业收入构成为：化学业务(WuXiChemistry)占比71.36%，测试业务(WuXiTesting)占比16.38%，生物学业务(WuXiBiology)占比6.53%，细胞及基因疗法CTDMO业务(WuXiATU)占比3.78%，国内新药研发服务部(WuXiDDSU)占比1.81%。\u3000\u3000药明康德的总裁、董事长均是Ge Li(李革)，男，56岁，学历背景为博士。\u3000\u3000截至发稿，药明康德市值为2333亿元。（文章来源：每日经济新闻）', 'items': [{'confidence': 0.835726, 'negative_prob': 0.0739234, 'positive_prob': 0.926077, 'sentiment': 2}], 'log_id': 1721905623474874377}


In [3]:
import concurrent.futures
# Process data using multithreading
def process_news(index, row):
    news_combine_content = row['News']
    cleaned_text = re.sub(r'\u3000', '', news_combine_content)
    response_result = Senta_Label_result(cleaned_text)
    time = 0
    while 'items' not in response_result:
        response_result = Senta_Label_result(cleaned_text)
        time += 1
        if time > 30:
            print(f"{index}:{response_result}")
            break
            
    if 'items' not in response_result:
        return index, response_result, -1, -1, -1
    
    sentiment_result = response_result['items'][0]
    if sentiment_result['sentiment'] == 2:
        sentiment = 1
    elif sentiment_result['sentiment'] == 0:
        sentiment = 0
    else:
        sentiment = "MID"
    confidence = sentiment_result['confidence']
    negative_prob = sentiment_result['negative_prob']
    positive_prob = sentiment_result['positive_prob']
    support = [confidence, negative_prob, positive_prob]
    if(index % 10 == 0):
        print(f"--Index: {index}, Label: {sentiment}, Support: {support}--")
    return index, sentiment, confidence, negative_prob, positive_prob


In [ ]:
# Calculate the size of each part
for i in range(15, 20):  # Left with 15
    print(f"----Training_set_part_{i + 1}.xlsx is labeling----")
    chunk = pd.read_excel(f'Training_dataset\\Training_set_labeled_with_Senta\\Training_set_part_{i + 1}.xlsx')

    # Create a new column "sentiment" to store sentiment labels
    result_df = chunk
    default_value = -1  #Default value = -1
    result_df['Sentiment'] = default_value
    result_df['Confidence'] = default_value
    result_df['Negative_prob'] = default_value
    result_df['Positive_prob'] = default_value

    sentiment_df = result_df
    # Set up a thread pool
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda x: process_news(*x), sentiment_df.iterrows()))
    # Write the results into the DataFrame
    for index, sentiment, confidence, negative_prob, positive_prob in results:
        sentiment_df.at[index, 'Sentiment'] = sentiment
        sentiment_df.at[index, 'Confidence'] = confidence
        sentiment_df.at[index, 'Negative_prob'] = negative_prob
        sentiment_df.at[index, 'Positive_prob'] = positive_prob
    sentiment_df.to_excel(f'Training_dataset\\Training_set_labeled_with_Senta\\Training_set_part_{i+1}.xlsx', index=False)
    print(f"------Training_set_part_{i+1}_saved------")


----Training_set_part_16.xlsx is labeling----
--Index: 0,Lable: 1,Support: [0.489198, 0.229861, 0.770139]--
--Index: 10,Lable: 1,Support: [0.955861, 0.0198625, 0.980138]--
--Index: 20,Lable: 1,Support: [0.997623, 0.00106978, 0.99893]--
--Index: 30,Lable: 1,Support: [0.975062, 0.011222, 0.988778]--
--Index: 40,Lable: 0,Support: [0.534137, 0.790361, 0.209639]--
--Index: 50,Lable: 0,Support: [0.280616, 0.676277, 0.323723]--
--Index: 60,Lable: 1,Support: [0.989244, 0.00484043, 0.99516]--
--Index: 70,Lable: 1,Support: [0.942251, 0.025987, 0.974013]--
--Index: 90,Lable: 0,Support: [0.9834, 0.99253, 0.0074701]--
--Index: 80,Lable: 1,Support: [0.987714, 0.00552857, 0.994471]--
--Index: 100,Lable: 1,Support: [0.989059, 0.00492351, 0.995077]--
--Index: 110,Lable: 0,Support: [0.108749, 0.598937, 0.401063]--
--Index: 120,Lable: 1,Support: [0.999179, 0.000369341, 0.999631]--
--Index: 130,Lable: 1,Support: [0.385898, 0.276346, 0.723654]--
--Index: 140,Lable: 1,Support: [0.177543, 0.370106, 0.629894]

--Index: 1280,Lable: 1,Support: [0.345814, 0.294384, 0.705616]--
--Index: 1270,Lable: 1,Support: [0.9938, 0.00278997, 0.99721]--
--Index: 1290,Lable: 1,Support: [0.979179, 0.0093693, 0.990631]--
--Index: 1300,Lable: 0,Support: [0.987335, 0.994301, 0.0056993]--
--Index: 1310,Lable: 0,Support: [0.920042, 0.964019, 0.0359812]--
--Index: 1320,Lable: 0,Support: [0.374473, 0.718513, 0.281487]--
--Index: 1330,Lable: 0,Support: [0.98448, 0.993016, 0.0069839]--
--Index: 1340,Lable: 1,Support: [0.0342874, 0.434571, 0.565429]--
--Index: 1350,Lable: 1,Support: [0.686329, 0.141152, 0.858848]--
--Index: 1360,Lable: 0,Support: [0.990994, 0.995947, 0.0040529]--
--Index: 1370,Lable: 0,Support: [0.203934, 0.64177, 0.35823]--
--Index: 1390,Lable: 0,Support: [0.453436, 0.754046, 0.245954]--
--Index: 1380,Lable: 0,Support: [0.201347, 0.640606, 0.359394]--
--Index: 1400,Lable: 1,Support: [0.70012, 0.134946, 0.865054]--
--Index: 1410,Lable: 0,Support: [0.354437, 0.709497, 0.290503]--
--Index: 1430,Lable: 1,S

--Index: 2510,Lable: MID,Support: [0.604387, 0.519781, 0.480219]--
--Index: 2520,Lable: 1,Support: [0.472896, 0.237197, 0.762803]--
--Index: 2530,Lable: 1,Support: [0.972862, 0.0122119, 0.987788]--
--Index: 2540,Lable: 1,Support: [0.820652, 0.0807066, 0.919293]--
--Index: 2550,Lable: MID,Support: [0.118227, 0.455911, 0.544089]--
--Index: 2560,Lable: 0,Support: [0.0859614, 0.588683, 0.411317]--
--Index: 2570,Lable: 1,Support: [0.928489, 0.0321798, 0.96782]--
--Index: 2580,Lable: 1,Support: [0.696116, 0.136748, 0.863252]--
--Index: 2590,Lable: 1,Support: [0.893482, 0.0479333, 0.952067]--
--Index: 2600,Lable: 0,Support: [0.2676, 0.67042, 0.32958]--
--Index: 2610,Lable: 1,Support: [0.772266, 0.10248, 0.89752]--
--Index: 2620,Lable: MID,Support: [0.262741, 0.536863, 0.463137]--
--Index: 2630,Lable: 0,Support: [0.720403, 0.874181, 0.125819]--
--Index: 2640,Lable: 1,Support: [0.787234, 0.0957447, 0.904255]--
--Index: 2650,Lable: 1,Support: [0.183021, 0.367641, 0.632359]--
--Index: 2660,Lable:

--Index: 3750,Lable: 1,Support: [0.96578, 0.0153992, 0.984601]--
--Index: 3760,Lable: 1,Support: [0.977169, 0.0102738, 0.989726]--
--Index: 3770,Lable: 0,Support: [0.953009, 0.978854, 0.021146]--
--Index: 3790,Lable: 1,Support: [0.936211, 0.028705, 0.971295]--
--Index: 3780,Lable: 0,Support: [0.829799, 0.923409, 0.0765907]--
--Index: 3800,Lable: 1,Support: [0.857864, 0.0639611, 0.936039]--
--Index: 3810,Lable: 1,Support: [0.593588, 0.182886, 0.817114]--
--Index: 3820,Lable: 1,Support: [0.92055, 0.0357524, 0.964248]--
--Index: 3830,Lable: MID,Support: [0.38678, 0.469339, 0.530661]--
--Index: 3840,Lable: 0,Support: [0.93705, 0.971672, 0.0283277]--
--Index: 3860,Lable: 1,Support: [0.363902, 0.286244, 0.713756]--
--Index: 3850,Lable: 1,Support: [0.673627, 0.146868, 0.853132]--
--Index: 3880,Lable: 1,Support: [0.756386, 0.109626, 0.890374]--
--Index: 3870,Lable: 0,Support: [0.453663, 0.754148, 0.245852]--
--Index: 3890,Lable: 1,Support: [0.951274, 0.0219265, 0.978073]--
--Index: 3900,Lable:

--Index: 10,Lable: 0,Support: [0.984974, 0.993239, 0.00676147]--
--Index: 0,Lable: 1,Support: [0.602522, 0.178865, 0.821135]--
--Index: 30,Lable: 1,Support: [0.769477, 0.103735, 0.896265]--


In [4]:
# Calculate the size of each part
full_list = []
for i in range(0, 20):
    print(f"----Training_set_part_{i + 1}.xlsx is concatting----")
    chunk = pd.read_excel(f'Training_dataset\\Training_set_labeled_with_Senta\\Training_set_part_{i + 1}.xlsx')
    full_list.append(chunk)
full_df = pd.concat(full_list, axis=0)
full_df.to_excel(f'Training_dataset\\Training_set_labeled_with_Senta\\Training_set_labeled_with_Senta.xlsx', index=False)
    

----Training_set_part_1.xlsx is concatting----
----Training_set_part_2.xlsx is concatting----
----Training_set_part_3.xlsx is concatting----
----Training_set_part_4.xlsx is concatting----
----Training_set_part_5.xlsx is concatting----
----Training_set_part_6.xlsx is concatting----
----Training_set_part_7.xlsx is concatting----
----Training_set_part_8.xlsx is concatting----
----Training_set_part_9.xlsx is concatting----
----Training_set_part_10.xlsx is concatting----
----Training_set_part_11.xlsx is concatting----
----Training_set_part_12.xlsx is concatting----
----Training_set_part_13.xlsx is concatting----
----Training_set_part_14.xlsx is concatting----
----Training_set_part_15.xlsx is concatting----
----Training_set_part_16.xlsx is concatting----
----Training_set_part_17.xlsx is concatting----
----Training_set_part_18.xlsx is concatting----
----Training_set_part_19.xlsx is concatting----
----Training_set_part_20.xlsx is concatting----


### 3.4. Label Crowdsource 

In [23]:
import pandas as pd
# Read annotation result by senta
annotation_with_Senta_file = 'Training_dataset\\Training_set_labeled_with_Senta.xlsx'
annotation_with_Senta = pd.read_excel(annotation_with_Senta_file)

# Read annotation result by ERNIE
annotation_with_ERNIE_file = 'Training_dataset\\Training_set_labeled_with_ERNIE-Bot-turbo.xlsx'
annotation_with_ERNIE = pd.read_excel(annotation_with_ERNIE_file)

#### 1) annotation_with_Senta

In [24]:
annotation_with_Senta.head()

,NewsID,Explicit_Company,News,Sentiment,Confidence,Negative_prob,Positive_prob
0,1,建设银行,建设银行原董事长张恩照一审被判15年 本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京市第一中级人民法院依法以受贿罪判处张恩照有期徒刑１５年。 法院经开庭审理查明，２０００年至２００４年期间，被告人张恩照利用其担任原中国建设银行副行长、行长，中国建设银行股份有限公司董事长的职务便利，为他人牟取利益，多次非法收受他人给予的款物共计人民币４００余万元。案发后，赃款、赃物已全部退缴。 法院认为，被告人张恩照身为国家工作人员，利用职务上的便利，为他人谋取利益，非法收受他人财物，其行为已构成受贿罪，受贿数额特别巨大。鉴于张恩照因其他违纪问题被审查后，主动交代了有关部门不掌握的本案受贿事实，属于自首，且赃款、赃物已全部退缴，对张恩照依法可从轻处罚。法院遂依法以受贿罪判处张恩照有期徒刑１５年。,0,0.995106,0.997798,0.002202
1,3,"中国国航, 外运发展",外运发展：价值型蓝筹股补涨要求强烈 在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重股股价高位运行的风险，目前主力资金主要针对价值低估的二线展开了挖掘，使得近期上演二线蓝筹轮番上涨行情，建议投资者对该品种重点关注，如（600270）外运发展，该股拥有显著的盈利能力，除权后缺口对股价具有较强的牵引作用，目前涨幅滞后，后市可重点关注。 外运龙头国际市场高占有率 公司核心业务包括航空货运代理、国际快件及综合物流，在全国拥有35家分公司，80个市级物流中心及近200个物流网点，运营网络辐射全国，同时，在国际上与50多家海外货运代理公司有着通力合作，目前在国际航空快件领域和国际航空货运代理领域分别拥有约40%和10%的市场占有率，公司力争发展成为既有航空运力保障，又有地面网络通道的国内一流的快速物流供应商。该公司主要利润来源是与DHL(德国敦豪)合资的中外运敦豪(双方各持50%)，中外运敦豪的资产规模为13亿元，06年上半年实现净利润32217万元(05年实现净利润42634万元、04年净利润39514万元)，中外运敦豪是DHL全球服务网络的重要组成部分，在中国主要城市设立了56家分公司(其中04年新增6家分公司)。根据DHL的规划，未来3年合资公司的利润复合增长率有望达到14%以上，有能力保证未来业绩增长。 货运航空将成新的利润增长点 公司参与四川航空集团公司整体改制重组，公司将占新设的四川航空集团有限责任公司注册资本的49%。参股新公司后，可为公司的航空货运代理业务以及航空快递业务提供运力支持，从而大大提高公司的竞争优势，此外06年9月，公司与大韩航空、韩亚投资和新韩投资在京签署了合资组建货运航空公司的正式协议，新组建公司注册资本为6500万美元，外运发展占股权的51%。合资公司将以全货机运营，主要经营国际定期与非定期航空货物运输服务，同时兼营自用飞行器租赁和维修服务、包机业务、航空公司间代理服务、进出口贸易服务及相关地面服务等，货运航空业务有望成为公司未来的业绩增长点。 资产增值概念后市强势上行 二级市场上，该股三季度业绩平稳增长每股受益高达0.42元，属于典型的高收益高分红的价值型蓝筹股，该股同时还具有资产增值概念，其购买的中国国航目前为公司带来亿元以上的帐面收益，技术上该股除权后留下巨大缺口对股价有一定的牵引作用，作为业绩优秀的二线蓝筹，后市有强烈补涨要求。,1,0.798077,0.090865,0.909135
2,4,胜利股份,"胜利股份：稳步走强形成标准上升通道 胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左右。公司旗下的青岛胜通海岸置业发展有限公司对青岛市四方区海岸带项目的开发，岸线全长5830米，开发后形成新陆域面积预计高达2800亩。子公司低成本的填海造地项目使公司土地猛然增百万平米，加之青岛地区优越的经济、旅游环境，土地增值空间非常巨大，以青岛的地价估算，静态价值在10亿元左右。项目竣工后，公司可直接转让该块土地，获得土地转让收入；或则和一些世界级公司合作，共同参与港口后续的建设、经营和管理，从而介入港口业务。目前因为该项目预期收益良好，已有深圳南山集团、新加坡港务集团、阿联酋迪拜港等国际投资者前来洽谈投资事宜。多种方式的选择也为公司从该项目获取巨大收益提供了空间，为公司资产带来巨大增值机会。 公司在生物医药产业也初露锋芒，研究开发的海南霉素作为中国唯一自主开发的抗生素兽药，列入国家""十五""科技攻关项目，具有广泛用途。同时，公司另一拳头产品泰妙菌素是一种双萜类动物专用抗生素，主要用于防治家畜家禽呼吸系统疾病和动物促生长，是世界十大兽用抗生素之一，公司出口量占国内出口总量的比重在80％左右。公司斥资3亿元建设的济宁生物工程项目去年10月正式试车投产，将成为以生物发酵工程为主的大型生产基地，预计年实现销售收入3.5亿元，利税5000万元，成为公司显著的利润增长点。 公司农药产业随着国家""三农""政策的实施，保持了稳定的上升势头。公司农药产品主要由山东胜邦绿野化学有限公司（胜邦绿野）负责生产，公司持有胜邦绿野85％的股权，胜邦绿野是国家农药定点生产企业，我国最大的除草剂生产基地之一，山东省外经委确认的""先进技术型企业""。公司原来生产能力为1.6万吨，今年利用厂区搬迁的机会，产能扩张50％以上，新增加8000吨草甘膦项目。由此可以看到，公司农药产业以章丘草甘膦项目投产、厂区扩建为契机，加快产品结构调整，加强国内外市场的营销力度，实现收入、利润的稳步增长。 该股股改后稳步走强，形成标准的上升通道，且量价配合十分理想。近日该股之后再次进入攻击格局，后市有望继续走强，投资者可重点关注。",1,0.978145,0.009835,0.990165
3,41,"中联重科, 柳工, 中国石化, 三一重工, 宝钢股份, 武钢股份, 保利地产, 安徽合力, 万科A, 山推股份, 金融街, 中国重汽, 北辰实业, 金地集团, 宇通客车, 江铃汽车",上证指数盘中摸高2100点 深成指距历史高点仅差400多点 证券时报记者 彭 松 本报讯 在蓝筹股整体走强的带动下，沪深两市以普涨之势再创本轮行情新高。上证指数盘中最高冲至2102.06点，收报2099.29点，涨幅达2.2％；深证成指则是大涨了3.84%，收报当天最高点5667点，距1997年高点6103.62点相差不过436.62点。两市成交量大幅放大，合计约714亿元，较前一交易日增逾四成。 大盘蓝筹股再度扮演领涨角色。其中中国石化大涨了5.14%，而宝钢股份、武钢股份也放量大涨，并带动整个钢铁板块集体上扬。二线蓝筹股也纷纷走强，汽车、工程机械、地产、白酒、化工等行业蓝筹品种均出现大幅上涨。封闭式基金昨日也走势强劲，上证基金指数大涨6.19%。 房地产板块昨日继续向上攀升。地产龙头股万科A罕见地放量涨停。交易公开信息显示，其前五位买入席位中，有三个为机构专用席位，其中两个席位为全部单向买入，单此两个席位合计净买入金额即1亿元。不过，在前五位卖出席位中，也有两个机构专用席位出现净卖出，但合计卖出金额仅约为4000万元。此外，北辰实业昨日也告涨停，而金地集团、保利地产、华侨城、金融街等蓝筹地产股也位居涨幅榜前列。 工程机械板块昨日也再显强者恒强风范。柳工、S徐工涨停，三一重工、中联重科、山推股份、安徽合力等大幅上涨。汽车板块再度发力，中国重汽、江铃汽车、上海汽车、宇通客车等也位居涨幅榜前列。 不少机构人士都表示，目前市场强势特征非常明显，短期内股指仍将继续向上攀升，而蓝筹股也将继续扮演领涨角色。,MID,0.084145,0.454207,0.545793
4,46,光大证券,"专家：中国经济的宏观调控明年仍需紧缩 明年中国经济的宏观调控是否应该放松？专家学者们出现了分歧。昨日，国家发改委宏观经济研究院的经济形势分析课题组公布研究报告称，住宅投资""高烧""等经济过热的诱因尚未消除，投资可能再次反弹，明年不能放松宏观调控。 不久前，汇丰银行、光大证券和中国人民大学经济研究所等机构提出了放松宏观调控的建议。但是，国家发改委宏观经济研究院的报告强调，目前金融体系流动性过剩问题依然突出，住宅投资过热和房价居高不下导致资源在产业之间配置极不合理，而相关改革尚未取得实质性进展，投资冲动可能重新抬头。 ""住宅投资仍然保持接近30%的高增长，而且是逆调控上扬。与此相对应的是，房价持续居高不下，增幅仍然偏高""。 统计数字显示，房屋销售价格结束了连续两个月增幅回落的趋势，到10月份，全国70个大中城市房屋销售价格比去年同期上涨5.4％，比上个月上涨0.4个百分点。报告分析称，这表明全国房地产投资过热现象还没有从根本上得到抑制，房地产调控的实际效果同预期目标之间仍存在一定差距。 房地产投资过热被报告视为一大核心问题。报告称，一些产能过剩的行业与房地产关系较大，后者持续""高烧""使这些产业的投资增长难以回落，产业结构调整难以推进。同时，房地产的高利润机制，促使过量的社会资金和资源向该行业集中。在主要金融机构新增的长期贷款中，房地产业所占比重高达20.2%。 此前，其他研究机构建议放松宏观调控的一大理由是，明年中国经济增长速度可能降低。国家发改委宏观经济研究院的报告则认为，宏观调控也是为了在中长期转变经济增长方式，即使调控导致经济转冷，那也是为实现这种历

In [25]:
# Counting 0/1/MID occurrences
sentiment_distribution_Senta = annotation_with_Senta['Sentiment'].value_counts()
sentiment_distribution_Senta = sentiment_distribution_Senta[:3]
# Adding a row to sentiment_distribution_Senta to count other value occurrences
# Counting occurrences of values besides 0/1/MID
other_values_count_Senta = len(annotation_with_Senta) - sentiment_distribution_Senta.sum()
# Adding a new row
sentiment_distribution_Senta.loc['Other'] = other_values_count_Senta

print(sentiment_distribution_Senta)


# Retrieving examples of values besides 0/1/MID
other_values_Senta = annotation_with_Senta[~annotation_with_Senta['Sentiment'].isin([0, 1, 'MID'])]['Sentiment'].sample(n=5, random_state=1)
pd.set_option('display.max_rows', None)  # Showing all rows
pd.set_option('display.max_colwidth', None)  # Showing content in all columns
print("\n Random samples of 'Other' values in 'Sentiment' column for Senta:")
print(other_values_Senta.to_string(index=False))

Sentiment
1        67361
0        26949
MID       5617
Other       73
Name: count, dtype: int64

 Random samples of 'Other' values in 'Sentiment' column for Senta:
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}


In [26]:
# Handling intermediate sentiment data
def replace_MID(row):
    if row['Sentiment'] in ['MID']:
        return 1 if row['Positive_prob'] > 0.5 else 0
    else:
        return row['Sentiment']
annotation_with_Senta['New_Sentiment'] = annotation_with_Senta.apply(replace_MID, axis=1)

# Marking missing values
def replace_other(row):
    if row['New_Sentiment'] in [0, 1, 'MID']:
        return row['New_Sentiment']
    else:
        return -1
annotation_with_Senta['New_Sentiment'] = annotation_with_Senta.apply(replace_other, axis=1)


# Count occurrences of 0/1/MID
sentiment_distribution_Senta = annotation_with_Senta['New_Sentiment'].value_counts()
sentiment_distribution_Senta = sentiment_distribution_Senta
print(sentiment_distribution_Senta)

New_Sentiment
 1    69844
 0    30083
-1       73
Name: count, dtype: int64


#### 2) annotation_with_ERNIE

In [27]:
annotation_with_ERNIE.head()

,NewsID,Explicit_Company,News,Sentiment
0,1,建设银行,建设银行原董事长张恩照一审被判15年 本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京市第一中级人民法院依法以受贿罪判处张恩照有期徒刑１５年。 法院经开庭审理查明，２０００年至２００４年期间，被告人张恩照利用其担任原中国建设银行副行长、行长，中国建设银行股份有限公司董事长的职务便利，为他人牟取利益，多次非法收受他人给予的款物共计人民币４００余万元。案发后，赃款、赃物已全部退缴。 法院认为，被告人张恩照身为国家工作人员，利用职务上的便利，为他人谋取利益，非法收受他人财物，其行为已构成受贿罪，受贿数额特别巨大。鉴于张恩照因其他违纪问题被审查后，主动交代了有关部门不掌握的本案受贿事实，属于自首，且赃款、赃物已全部退缴，对张恩照依法可从轻处罚。法院遂依法以受贿罪判处张恩照有期徒刑１５年。,0
1,3,"中国国航, 外运发展",外运发展：价值型蓝筹股补涨要求强烈 在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重股股价高位运行的风险，目前主力资金主要针对价值低估的二线展开了挖掘，使得近期上演二线蓝筹轮番上涨行情，建议投资者对该品种重点关注，如（600270）外运发展，该股拥有显著的盈利能力，除权后缺口对股价具有较强的牵引作用，目前涨幅滞后，后市可重点关注。 外运龙头国际市场高占有率 公司核心业务包括航空货运代理、国际快件及综合物流，在全国拥有35家分公司，80个市级物流中心及近200个物流网点，运营网络辐射全国，同时，在国际上与50多家海外货运代理公司有着通力合作，目前在国际航空快件领域和国际航空货运代理领域分别拥有约40%和10%的市场占有率，公司力争发展成为既有航空运力保障，又有地面网络通道的国内一流的快速物流供应商。该公司主要利润来源是与DHL(德国敦豪)合资的中外运敦豪(双方各持50%)，中外运敦豪的资产规模为13亿元，06年上半年实现净利润32217万元(05年实现净利润42634万元、04年净利润39514万元)，中外运敦豪是DHL全球服务网络的重要组成部分，在中国主要城市设立了56家分公司(其中04年新增6家分公司)。根据DHL的规划，未来3年合资公司的利润复合增长率有望达到14%以上，有能力保证未来业绩增长。 货运航空将成新的利润增长点 公司参与四川航空集团公司整体改制重组，公司将占新设的四川航空集团有限责任公司注册资本的49%。参股新公司后，可为公司的航空货运代理业务以及航空快递业务提供运力支持，从而大大提高公司的竞争优势，此外06年9月，公司与大韩航空、韩亚投资和新韩投资在京签署了合资组建货运航空公司的正式协议，新组建公司注册资本为6500万美元，外运发展占股权的51%。合资公司将以全货机运营，主要经营国际定期与非定期航空货物运输服务，同时兼营自用飞行器租赁和维修服务、包机业务、航空公司间代理服务、进出口贸易服务及相关地面服务等，货运航空业务有望成为公司未来的业绩增长点。 资产增值概念后市强势上行 二级市场上，该股三季度业绩平稳增长每股受益高达0.42元，属于典型的高收益高分红的价值型蓝筹股，该股同时还具有资产增值概念，其购买的中国国航目前为公司带来亿元以上的帐面收益，技术上该股除权后留下巨大缺口对股价有一定的牵引作用，作为业绩优秀的二线蓝筹，后市有强烈补涨要求。,1
2,4,胜利股份,"胜利股份：稳步走强形成标准上升通道 胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左右。公司旗下的青岛胜通海岸置业发展有限公司对青岛市四方区海岸带项目的开发，岸线全长5830米，开发后形成新陆域面积预计高达2800亩。子公司低成本的填海造地项目使公司土地猛然增百万平米，加之青岛地区优越的经济、旅游环境，土地增值空间非常巨大，以青岛的地价估算，静态价值在10亿元左右。项目竣工后，公司可直接转让该块土地，获得土地转让收入；或则和一些世界级公司合作，共同参与港口后续的建设、经营和管理，从而介入港口业务。目前因为该项目预期收益良好，已有深圳南山集团、新加坡港务集团、阿联酋迪拜港等国际投资者前来洽谈投资事宜。多种方式的选择也为公司从该项目获取巨大收益提供了空间，为公司资产带来巨大增值机会。 公司在生物医药产业也初露锋芒，研究开发的海南霉素作为中国唯一自主开发的抗生素兽药，列入国家""十五""科技攻关项目，具有广泛用途。同时，公司另一拳头产品泰妙菌素是一种双萜类动物专用抗生素，主要用于防治家畜家禽呼吸系统疾病和动物促生长，是世界十大兽用抗生素之一，公司出口量占国内出口总量的比重在80％左右。公司斥资3亿元建设的济宁生物工程项目去年10月正式试车投产，将成为以生物发酵工程为主的大型生产基地，预计年实现销售收入3.5亿元，利税5000万元，成为公司显著的利润增长点。 公司农药产业随着国家""三农""政策的实施，保持了稳定的上升势头。公司农药产品主要由山东胜邦绿野化学有限公司（胜邦绿野）负责生产，公司持有胜邦绿野85％的股权，胜邦绿野是国家农药定点生产企业，我国最大的除草剂生产基地之一，山东省外经委确认的""先进技术型企业""。公司原来生产能力为1.6万吨，今年利用厂区搬迁的机会，产能扩张50％以上，新增加8000吨草甘膦项目。由此可以看到，公司农药产业以章丘草甘膦项目投产、厂区扩建为契机，加快产品结构调整，加强国内外市场的营销力度，实现收入、利润的稳步增长。 该股股改后稳步走强，形成标准的上升通道，且量价配合十分理想。近日该股之后再次进入攻击格局，后市有望继续走强，投资者可重点关注。",1
3,41,"中联重科, 柳工, 中国石化, 三一重工, 宝钢股份, 武钢股份, 保利地产, 安徽合力, 万科A, 山推股份, 金融街, 中国重汽, 北辰实业, 金地集团, 宇通客车, 江铃汽车",上证指数盘中摸高2100点 深成指距历史高点仅差400多点 证券时报记者 彭 松 本报讯 在蓝筹股整体走强的带动下，沪深两市以普涨之势再创本轮行情新高。上证指数盘中最高冲至2102.06点，收报2099.29点，涨幅达2.2％；深证成指则是大涨了3.84%，收报当天最高点5667点，距1997年高点6103.62点相差不过436.62点。两市成交量大幅放大，合计约714亿元，较前一交易日增逾四成。 大盘蓝筹股再度扮演领涨角色。其中中国石化大涨了5.14%，而宝钢股份、武钢股份也放量大涨，并带动整个钢铁板块集体上扬。二线蓝筹股也纷纷走强，汽车、工程机械、地产、白酒、化工等行业蓝筹品种均出现大幅上涨。封闭式基金昨日也走势强劲，上证基金指数大涨6.19%。 房地产板块昨日继续向上攀升。地产龙头股万科A罕见地放量涨停。交易公开信息显示，其前五位买入席位中，有三个为机构专用席位，其中两个席位为全部单向买入，单此两个席位合计净买入金额即1亿元。不过，在前五位卖出席位中，也有两个机构专用席位出现净卖出，但合计卖出金额仅约为4000万元。此外，北辰实业昨日也告涨停，而金地集团、保利地产、华侨城、金融街等蓝筹地产股也位居涨幅榜前列。 工程机械板块昨日也再显强者恒强风范。柳工、S徐工涨停，三一重工、中联重科、山推股份、安徽合力等大幅上涨。汽车板块再度发力，中国重汽、江铃汽车、上海汽车、宇通客车等也位居涨幅榜前列。 不少机构人士都表示，目前市场强势特征非常明显，短期内股指仍将继续向上攀升，而蓝筹股也将继续扮演领涨角色。,1
4,46,光大证券,"专家：中国经济的宏观调控明年仍需紧缩 明年中国经济的宏观调控是否应该放松？专家学者们出现了分歧。昨日，国家发改委宏观经济研究院的经济形势分析课题组公布研究报告称，住宅投资""高烧""等经济过热的诱因尚未消除，投资可能再次反弹，明年不能放松宏观调控。 不久前，汇丰银行、光大证券和中国人民大学经济研究所等机构提出了放松宏观调控的建议。但是，国家发改委宏观经济研究院的报告强调，目前金融体系流动性过剩问题依然突出，住宅投资过热和房价居高不下导致资源在产业之间配置极不合理，而相关改革尚未取得实质性进展，投资冲动可能重新抬头。 ""住宅投资仍然保持接近30%的高增长，而且是逆调控上扬。与此相对应的是，房价持续居高不下，增幅仍然偏高""。 统计数字显示，房屋销售价格结束了连续两个月增幅回落的趋势，到10月份，全国70个大中城市房屋销售价格比去年同期上涨5.4％，比上个月上涨0.4个百分点。报告分析称，这表明全国房地产投资过热现象还没有从根本上得到抑制，房地产调控的实际效果同预期目标之间仍存在一定差距。 房地产投资过热被报告视为一大核心问题。报告称，一些产能过剩的行业与房地产关系较大，后者持续""高烧""使这些产业的投资增长难以回落，产业结构调整难以推进。同时，房地产的高利润机制，促使过量的社会资金和资源向该行业集中。在主要金融机构新增的长期贷款中，房地产业所占比重高达20.2%。 此前，其他研究机构建议放松宏观调控的一大理由是，明年中国经济增长速度可能降低。国家发改委宏观经济研究院的报告则认为，宏观调控也是为了在中长期转变经济增长方式，即使调控导致经济转冷，那也是为实现这种历史性转变所应付出的代价。如果见热则压、遇冷则放，一味追求经济短期的平滑运行，是典型的治标不治本。况且政策的摇摆性是调控的大忌，不但不能给市场稳定的政策预期，而且会导致中央与地方之间的关系随意化，甚至产生对立情绪。（北京晨报）",0


In [28]:
# Count occurrences of 0/1
sentiment_distribution_ERNIE = annotation_with_ERNIE['Sentiment'].value_counts()
sentiment_distribution_ERNIE = sentiment_distribution_ERNIE[:2]

# Add a row to sentiment_distribution_ERNIE, counting the occurrences of other values
# Count occurrences of values other than 0/1
other_values_count_ERNIE = len(annotation_with_ERNIE) - sentiment_distribution_ERNIE.sum()
# Add a new row
sentiment_distribution_ERNIE.loc['Other'] = other_values_count_ERNIE

print(sentiment_distribution_ERNIE)


# Obtain examples of values other than 0/1
other_values_ERNIE = annotation_with_ERNIE[~annotation_with_ERNIE['Sentiment'].isin([0, 1])]['Sentiment'].sample(n=5, random_state=1)
pd.set_option('display.max_rows', None)  # Set to display all rows
pd.set_option('display.max_colwidth', None)  # Set to display all content in columns
print("\n Random samples of 'Other' values in 'Sentiment' column for ERNIE:")
print(other_values_ERNIE.to_string(index=False))

Sentiment
1        85456
0        14469
Other       75
Name: count, dtype: int64

 Random samples of 'Other' values in 'Sentiment' column for ERNIE:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          {"id":"as-3gh6cvver4","object":"chat.completion","created":1699194792,"result":"根据相关法律，这个问题不予以回答。您可以问我一些其它问题，我会尽力为您解答。","is_truncated":false,"need_clear_history":true,"ban_round":-1,"usage":{"prompt_tokens":420,"completion_tokens":34

In [29]:
# Missing value labeling
def replace_other(row):
    if row['Sentiment'] in [0, 1]:
        return row['Sentiment']
    else:
        return -1
annotation_with_ERNIE['New_Sentiment'] = annotation_with_ERNIE.apply(replace_other, axis=1)


# Count occurrences of 0/1/missing
sentiment_distribution_ERNIE = annotation_with_ERNIE['New_Sentiment'].value_counts()
sentiment_distribution_ERNIE = sentiment_distribution_ERNIE
print(sentiment_distribution_ERNIE)

New_Sentiment
 1    85456
 0    14469
-1       75
Name: count, dtype: int64


#### 3) final_annotation

In [30]:
sentiment_distribution_ERNIE = annotation_with_ERNIE['New_Sentiment'].value_counts()
sentiment_distribution_ERNIE = sentiment_distribution_ERNIE
print(sentiment_distribution_ERNIE)

sentiment_distribution_Senta = annotation_with_Senta['New_Sentiment'].value_counts()
sentiment_distribution_Senta = sentiment_distribution_Senta
print(sentiment_distribution_Senta)

New_Sentiment
 1    85456
 0    14469
-1       75
Name: count, dtype: int64
New_Sentiment
 1    69844
 0    30083
-1       73
Name: count, dtype: int64


In [31]:
# 1. Check if the NewsID column corresponds one-to-one
news_id_comparison = annotation_with_ERNIE['NewsID'].equals(annotation_with_Senta['NewsID'])
print("NewsID columns match:", news_id_comparison)

# 2. Create annotation_final table and copy data from annotation_with_ERNIE
annotation_final = annotation_with_ERNIE.copy()

# 3. Merge specified columns from annotation_with_Senta into annotation_final and modify column names
merge_columns = ['New_Sentiment', 'Confidence', 'Negative_prob', 'Positive_prob']
merged_data = annotation_with_Senta.loc[:, merge_columns]
merged_data.columns = [col + '_Senta' for col in merged_data.columns]  # Rename columns

annotation_final = pd.concat([annotation_final, merged_data], axis=1)
annotation_final.drop(columns='Sentiment', inplace=True)

NewsID columns match: True


In [32]:
annotation_final.head()

,NewsID,Explicit_Company,News,New_Sentiment,New_Sentiment_Senta,Confidence_Senta,Negative_prob_Senta,Positive_prob_Senta
0,1,建设银行,建设银行原董事长张恩照一审被判15年 本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京市第一中级人民法院依法以受贿罪判处张恩照有期徒刑１５年。 法院经开庭审理查明，２０００年至２００４年期间，被告人张恩照利用其担任原中国建设银行副行长、行长，中国建设银行股份有限公司董事长的职务便利，为他人牟取利益，多次非法收受他人给予的款物共计人民币４００余万元。案发后，赃款、赃物已全部退缴。 法院认为，被告人张恩照身为国家工作人员，利用职务上的便利，为他人谋取利益，非法收受他人财物，其行为已构成受贿罪，受贿数额特别巨大。鉴于张恩照因其他违纪问题被审查后，主动交代了有关部门不掌握的本案受贿事实，属于自首，且赃款、赃物已全部退缴，对张恩照依法可从轻处罚。法院遂依法以受贿罪判处张恩照有期徒刑１５年。,0,0,0.995106,0.997798,0.002202
1,3,"中国国航, 外运发展",外运发展：价值型蓝筹股补涨要求强烈 在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重股股价高位运行的风险，目前主力资金主要针对价值低估的二线展开了挖掘，使得近期上演二线蓝筹轮番上涨行情，建议投资者对该品种重点关注，如（600270）外运发展，该股拥有显著的盈利能力，除权后缺口对股价具有较强的牵引作用，目前涨幅滞后，后市可重点关注。 外运龙头国际市场高占有率 公司核心业务包括航空货运代理、国际快件及综合物流，在全国拥有35家分公司，80个市级物流中心及近200个物流网点，运营网络辐射全国，同时，在国际上与50多家海外货运代理公司有着通力合作，目前在国际航空快件领域和国际航空货运代理领域分别拥有约40%和10%的市场占有率，公司力争发展成为既有航空运力保障，又有地面网络通道的国内一流的快速物流供应商。该公司主要利润来源是与DHL(德国敦豪)合资的中外运敦豪(双方各持50%)，中外运敦豪的资产规模为13亿元，06年上半年实现净利润32217万元(05年实现净利润42634万元、04年净利润39514万元)，中外运敦豪是DHL全球服务网络的重要组成部分，在中国主要城市设立了56家分公司(其中04年新增6家分公司)。根据DHL的规划，未来3年合资公司的利润复合增长率有望达到14%以上，有能力保证未来业绩增长。 货运航空将成新的利润增长点 公司参与四川航空集团公司整体改制重组，公司将占新设的四川航空集团有限责任公司注册资本的49%。参股新公司后，可为公司的航空货运代理业务以及航空快递业务提供运力支持，从而大大提高公司的竞争优势，此外06年9月，公司与大韩航空、韩亚投资和新韩投资在京签署了合资组建货运航空公司的正式协议，新组建公司注册资本为6500万美元，外运发展占股权的51%。合资公司将以全货机运营，主要经营国际定期与非定期航空货物运输服务，同时兼营自用飞行器租赁和维修服务、包机业务、航空公司间代理服务、进出口贸易服务及相关地面服务等，货运航空业务有望成为公司未来的业绩增长点。 资产增值概念后市强势上行 二级市场上，该股三季度业绩平稳增长每股受益高达0.42元，属于典型的高收益高分红的价值型蓝筹股，该股同时还具有资产增值概念，其购买的中国国航目前为公司带来亿元以上的帐面收益，技术上该股除权后留下巨大缺口对股价有一定的牵引作用，作为业绩优秀的二线蓝筹，后市有强烈补涨要求。,1,1,0.798077,0.090865,0.909135
2,4,胜利股份,"胜利股份：稳步走强形成标准上升通道 胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左右。公司旗下的青岛胜通海岸置业发展有限公司对青岛市四方区海岸带项目的开发，岸线全长5830米，开发后形成新陆域面积预计高达2800亩。子公司低成本的填海造地项目使公司土地猛然增百万平米，加之青岛地区优越的经济、旅游环境，土地增值空间非常巨大，以青岛的地价估算，静态价值在10亿元左右。项目竣工后，公司可直接转让该块土地，获得土地转让收入；或则和一些世界级公司合作，共同参与港口后续的建设、经营和管理，从而介入港口业务。目前因为该项目预期收益良好，已有深圳南山集团、新加坡港务集团、阿联酋迪拜港等国际投资者前来洽谈投资事宜。多种方式的选择也为公司从该项目获取巨大收益提供了空间，为公司资产带来巨大增值机会。 公司在生物医药产业也初露锋芒，研究开发的海南霉素作为中国唯一自主开发的抗生素兽药，列入国家""十五""科技攻关项目，具有广泛用途。同时，公司另一拳头产品泰妙菌素是一种双萜类动物专用抗生素，主要用于防治家畜家禽呼吸系统疾病和动物促生长，是世界十大兽用抗生素之一，公司出口量占国内出口总量的比重在80％左右。公司斥资3亿元建设的济宁生物工程项目去年10月正式试车投产，将成为以生物发酵工程为主的大型生产基地，预计年实现销售收入3.5亿元，利税5000万元，成为公司显著的利润增长点。 公司农药产业随着国家""三农""政策的实施，保持了稳定的上升势头。公司农药产品主要由山东胜邦绿野化学有限公司（胜邦绿野）负责生产，公司持有胜邦绿野85％的股权，胜邦绿野是国家农药定点生产企业，我国最大的除草剂生产基地之一，山东省外经委确认的""先进技术型企业""。公司原来生产能力为1.6万吨，今年利用厂区搬迁的机会，产能扩张50％以上，新增加8000吨草甘膦项目。由此可以看到，公司农药产业以章丘草甘膦项目投产、厂区扩建为契机，加快产品结构调整，加强国内外市场的营销力度，实现收入、利润的稳步增长。 该股股改后稳步走强，形成标准的上升通道，且量价配合十分理想。近日该股之后再次进入攻击格局，后市有望继续走强，投资者可重点关注。",1,1,0.978145,0.009835,0.990165
3,41,"中联重科, 柳工, 中国石化, 三一重工, 宝钢股份, 武钢股份, 保利地产, 安徽合力, 万科A, 山推股份, 金融街, 中国重汽, 北辰实业, 金地集团, 宇通客车, 江铃汽车",上证指数盘中摸高2100点 深成指距历史高点仅差400多点 证券时报记者 彭 松 本报讯 在蓝筹股整体走强的带动下，沪深两市以普涨之势再创本轮行情新高。上证指数盘中最高冲至2102.06点，收报2099.29点，涨幅达2.2％；深证成指则是大涨了3.84%，收报当天最高点5667点，距1997年高点6103.62点相差不过436.62点。两市成交量大幅放大，合计约714亿元，较前一交易日增逾四成。 大盘蓝筹股再度扮演领涨角色。其中中国石化大涨了5.14%，而宝钢股份、武钢股份也放量大涨，并带动整个钢铁板块集体上扬。二线蓝筹股也纷纷走强，汽车、工程机械、地产、白酒、化工等行业蓝筹品种均出现大幅上涨。封闭式基金昨日也走势强劲，上证基金指数大涨6.19%。 房地产板块昨日继续向上攀升。地产龙头股万科A罕见地放量涨停。交易公开信息显示，其前五位买入席位中，有三个为机构专用席位，其中两个席位为全部单向买入，单此两个席位合计净买入金额即1亿元。不过，在前五位卖出席位中，也有两个机构专用席位出现净卖出，但合计卖出金额仅约为4000万元。此外，北辰实业昨日也告涨停，而金地集团、保利地产、华侨城、金融街等蓝筹地产股也位居涨幅榜前列。 工程机械板块昨日也再显强者恒强风范。柳工、S徐工涨停，三一重工、中联重科、山推股份、安徽合力等大幅上涨。汽车板块再度发力，中国重汽、江铃汽车、上海汽车、宇通客车等也位居涨幅榜前列。 不少机构人士都表示，目前市场强势特征非常明显，短期内股指仍将继续向上攀升，而蓝筹股也将继续扮演领涨角色。,1,1,0.084145,0.454207,0.545793
4,46,光大证券,"专家：中国经济的宏观调控明年仍需紧缩 明年中国经济的宏观调控是否应该放松？专家学者们出现了分歧。昨日，国家发改委宏观经济研究院的经济形势分析课题组公布研究报告称，住宅投资""高烧""等经济过热的诱因尚未消除，投资可能再次反弹，明年不能放松宏观调控。 不久前，汇丰银行、光大证券和中国人民大学经济研究所等机构提出了放松宏观调控的建议。但是，国家发改委宏观经济研究院的报告强调，目前金融体系流动性过剩问题依然突出，住宅投资过热和房价居高不下导致资源在产业之间配置极不合理，而相关改革尚未取得实质性进展，投资冲动可能重新抬头。 ""住宅投资仍然保持接近30%的高增长，而且是逆调控上扬。与此相对应的是，房价持续居高不下，增幅仍然偏高""。 统计数字显示，房屋销售价格结束了连续两个月增幅回落的趋势，到10月份，全国70个大中城市房屋销售价格比去年同期上涨5.4％，比上个月上涨0.4个百分点。报告分析称，这表明全国房地产投资过热现象还没有从根本上得到抑制，房地产调控的实际效果同预期目标之间仍存在一定差距。 房地产投资过热被报告视为一大核心问题。报告称，一些产能过剩的行业与房地产关系较大，后者持续""高烧""使这些产业的投资增长难以回落，产业结构调整难以推进。同时，房地产的高利润机制，促使过量的社会资金和资源向该行业集中。在主要金融机构新增的长期贷款中，房地产业所占比重高达20.2%。 此前，其他研究机构建议放松宏观调控的一大理由是，明年中国经济增长速度可能降低。国家发改委宏观经济研究院

In [33]:
def determine_final_sentiment(row):
    if row['New_Sentiment'] == row['New_Sentiment_Senta']:#若相同，则在新列Final_Sentiment保存该相同的值
        if row['New_Sentiment'] == -1:
            return pd.Series([row['New_Sentiment'], 'both_missing'])
        else:
            return pd.Series([row['New_Sentiment'], 'consensus'])
    elif row['New_Sentiment'] == -1 and row['New_Sentiment_Senta'] != -1:#若不同：New_Sentiment为-1，New_Sentiment_Senta不为-1
        return pd.Series([row['New_Sentiment_Senta'], 'ERNIE_missing_trust_Senta'])
    elif row['New_Sentiment_Senta'] == -1 and row['New_Sentiment'] != -1:#若不同：New_Sentiment_Senta为-1，New_Sentiment不为-1
        return pd.Series([row['New_Sentiment'], 'Senta_missing_trust_ERNIE'])
    else:#若不同：New_Sentiment_Senta和New_Sentiment都不为-1且不同
        if row['Confidence_Senta'] > 0.65:
            return pd.Series([row['New_Sentiment_Senta'], 'difference_trust_Senta'])
        else:
            return pd.Series([row['New_Sentiment'], 'difference_trust_ERNIE'])

# 创建新列 Final_Sentiment 和 Annotation_State
annotation_final[['Final_Sentiment', 'Annotation_State']] = annotation_final.apply(determine_final_sentiment, axis=1)

In [34]:
annotation_final.drop(columns=['New_Sentiment','New_Sentiment_Senta','Negative_prob_Senta','Positive_prob_Senta'], inplace=True)

In [35]:
annotation_final.head()

,NewsID,Explicit_Company,News,Confidence_Senta,Final_Sentiment,Annotation_State
0,1,建设银行,建设银行原董事长张恩照一审被判15年 本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京市第一中级人民法院依法以受贿罪判处张恩照有期徒刑１５年。 法院经开庭审理查明，２０００年至２００４年期间，被告人张恩照利用其担任原中国建设银行副行长、行长，中国建设银行股份有限公司董事长的职务便利，为他人牟取利益，多次非法收受他人给予的款物共计人民币４００余万元。案发后，赃款、赃物已全部退缴。 法院认为，被告人张恩照身为国家工作人员，利用职务上的便利，为他人谋取利益，非法收受他人财物，其行为已构成受贿罪，受贿数额特别巨大。鉴于张恩照因其他违纪问题被审查后，主动交代了有关部门不掌握的本案受贿事实，属于自首，且赃款、赃物已全部退缴，对张恩照依法可从轻处罚。法院遂依法以受贿罪判处张恩照有期徒刑１５年。,0.995106,0,consensus
1,3,"中国国航, 外运发展",外运发展：价值型蓝筹股补涨要求强烈 在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重股股价高位运行的风险，目前主力资金主要针对价值低估的二线展开了挖掘，使得近期上演二线蓝筹轮番上涨行情，建议投资者对该品种重点关注，如（600270）外运发展，该股拥有显著的盈利能力，除权后缺口对股价具有较强的牵引作用，目前涨幅滞后，后市可重点关注。 外运龙头国际市场高占有率 公司核心业务包括航空货运代理、国际快件及综合物流，在全国拥有35家分公司，80个市级物流中心及近200个物流网点，运营网络辐射全国，同时，在国际上与50多家海外货运代理公司有着通力合作，目前在国际航空快件领域和国际航空货运代理领域分别拥有约40%和10%的市场占有率，公司力争发展成为既有航空运力保障，又有地面网络通道的国内一流的快速物流供应商。该公司主要利润来源是与DHL(德国敦豪)合资的中外运敦豪(双方各持50%)，中外运敦豪的资产规模为13亿元，06年上半年实现净利润32217万元(05年实现净利润42634万元、04年净利润39514万元)，中外运敦豪是DHL全球服务网络的重要组成部分，在中国主要城市设立了56家分公司(其中04年新增6家分公司)。根据DHL的规划，未来3年合资公司的利润复合增长率有望达到14%以上，有能力保证未来业绩增长。 货运航空将成新的利润增长点 公司参与四川航空集团公司整体改制重组，公司将占新设的四川航空集团有限责任公司注册资本的49%。参股新公司后，可为公司的航空货运代理业务以及航空快递业务提供运力支持，从而大大提高公司的竞争优势，此外06年9月，公司与大韩航空、韩亚投资和新韩投资在京签署了合资组建货运航空公司的正式协议，新组建公司注册资本为6500万美元，外运发展占股权的51%。合资公司将以全货机运营，主要经营国际定期与非定期航空货物运输服务，同时兼营自用飞行器租赁和维修服务、包机业务、航空公司间代理服务、进出口贸易服务及相关地面服务等，货运航空业务有望成为公司未来的业绩增长点。 资产增值概念后市强势上行 二级市场上，该股三季度业绩平稳增长每股受益高达0.42元，属于典型的高收益高分红的价值型蓝筹股，该股同时还具有资产增值概念，其购买的中国国航目前为公司带来亿元以上的帐面收益，技术上该股除权后留下巨大缺口对股价有一定的牵引作用，作为业绩优秀的二线蓝筹，后市有强烈补涨要求。,0.798077,1,consensus
2,4,胜利股份,"胜利股份：稳步走强形成标准上升通道 胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左右。公司旗下的青岛胜通海岸置业发展有限公司对青岛市四方区海岸带项目的开发，岸线全长5830米，开发后形成新陆域面积预计高达2800亩。子公司低成本的填海造地项目使公司土地猛然增百万平米，加之青岛地区优越的经济、旅游环境，土地增值空间非常巨大，以青岛的地价估算，静态价值在10亿元左右。项目竣工后，公司可直接转让该块土地，获得土地转让收入；或则和一些世界级公司合作，共同参与港口后续的建设、经营和管理，从而介入港口业务。目前因为该项目预期收益良好，已有深圳南山集团、新加坡港务集团、阿联酋迪拜港等国际投资者前来洽谈投资事宜。多种方式的选择也为公司从该项目获取巨大收益提供了空间，为公司资产带来巨大增值机会。 公司在生物医药产业也初露锋芒，研究开发的海南霉素作为中国唯一自主开发的抗生素兽药，列入国家""十五""科技攻关项目，具有广泛用途。同时，公司另一拳头产品泰妙菌素是一种双萜类动物专用抗生素，主要用于防治家畜家禽呼吸系统疾病和动物促生长，是世界十大兽用抗生素之一，公司出口量占国内出口总量的比重在80％左右。公司斥资3亿元建设的济宁生物工程项目去年10月正式试车投产，将成为以生物发酵工程为主的大型生产基地，预计年实现销售收入3.5亿元，利税5000万元，成为公司显著的利润增长点。 公司农药产业随着国家""三农""政策的实施，保持了稳定的上升势头。公司农药产品主要由山东胜邦绿野化学有限公司（胜邦绿野）负责生产，公司持有胜邦绿野85％的股权，胜邦绿野是国家农药定点生产企业，我国最大的除草剂生产基地之一，山东省外经委确认的""先进技术型企业""。公司原来生产能力为1.6万吨，今年利用厂区搬迁的机会，产能扩张50％以上，新增加8000吨草甘膦项目。由此可以看到，公司农药产业以章丘草甘膦项目投产、厂区扩建为契机，加快产品结构调整，加强国内外市场的营销力度，实现收入、利润的稳步增长。 该股股改后稳步走强，形成标准的上升通道，且量价配合十分理想。近日该股之后再次进入攻击格局，后市有望继续走强，投资者可重点关注。",0.978145,1,consensus
3,41,"中联重科, 柳工, 中国石化, 三一重工, 宝钢股份, 武钢股份, 保利地产, 安徽合力, 万科A, 山推股份, 金融街, 中国重汽, 北辰实业, 金地集团, 宇通客车, 江铃汽车",上证指数盘中摸高2100点 深成指距历史高点仅差400多点 证券时报记者 彭 松 本报讯 在蓝筹股整体走强的带动下，沪深两市以普涨之势再创本轮行情新高。上证指数盘中最高冲至2102.06点，收报2099.29点，涨幅达2.2％；深证成指则是大涨了3.84%，收报当天最高点5667点，距1997年高点6103.62点相差不过436.62点。两市成交量大幅放大，合计约714亿元，较前一交易日增逾四成。 大盘蓝筹股再度扮演领涨角色。其中中国石化大涨了5.14%，而宝钢股份、武钢股份也放量大涨，并带动整个钢铁板块集体上扬。二线蓝筹股也纷纷走强，汽车、工程机械、地产、白酒、化工等行业蓝筹品种均出现大幅上涨。封闭式基金昨日也走势强劲，上证基金指数大涨6.19%。 房地产板块昨日继续向上攀升。地产龙头股万科A罕见地放量涨停。交易公开信息显示，其前五位买入席位中，有三个为机构专用席位，其中两个席位为全部单向买入，单此两个席位合计净买入金额即1亿元。不过，在前五位卖出席位中，也有两个机构专用席位出现净卖出，但合计卖出金额仅约为4000万元。此外，北辰实业昨日也告涨停，而金地集团、保利地产、华侨城、金融街等蓝筹地产股也位居涨幅榜前列。 工程机械板块昨日也再显强者恒强风范。柳工、S徐工涨停，三一重工、中联重科、山推股份、安徽合力等大幅上涨。汽车板块再度发力，中国重汽、江铃汽车、上海汽车、宇通客车等也位居涨幅榜前列。 不少机构人士都表示，目前市场强势特征非常明显，短期内股指仍将继续向上攀升，而蓝筹股也将继续扮演领涨角色。,0.084145,1,consensus
4,46,光大证券,"专家：中国经济的宏观调控明年仍需紧缩 明年中国经济的宏观调控是否应该放松？专家学者们出现了分歧。昨日，国家发改委宏观经济研究院的经济形势分析课题组公布研究报告称，住宅投资""高烧""等经济过热的诱因尚未消除，投资可能再次反弹，明年不能放松宏观调控。 不久前，汇丰银行、光大证券和中国人民大学经济研究所等机构提出了放松宏观调控的建议。但是，国家发改委宏观经济研究院的报告强调，目前金融体系流动性过剩问题依然突出，住宅投资过热和房价居高不下导致资源在产业之间配置极不合理，而相关改革尚未取得实质性进展，投资冲动可能重新抬头。 ""住宅投资仍然保持接近30%的高增长，而且是逆调控上扬。与此相对应的是，房价持续居高不下，增幅仍然偏高""。 统计数字显示，房屋销售价格结束了连续两个月增幅回落的趋势，到10月份，全国70个大中城市房屋销售价格比去年同期上涨5.4％，比上个月上涨0.4个百分点。报告分析称，这表明全国房地产投资过热现象还没有从根本上得到抑制，房地产调控的实际效果同预期目标之间仍存在一定差距。 房地产投资过热被报告视为一大核心问题。报告称，一些产能过剩的行业与房地产关系较大，后者持续""高烧""使这些产业的投资增长难以回落，产业结构调整难以推进。同时，房地产的高利润机制，促使过量的社会资金和资源向该行业集中。在主要金融机构新增的长期贷款中，房地产业所占比重高达20.2%。 此前，其他研究机构建议放松宏观调控的一大理由是，明年中国经济增长速度可能降低。国家发改委宏观经济研究院的报告则认为，宏观调控也是为了在中长期转变经济增长方式，即使调控导致经济转冷，那也是为实现这种历史性转变所应付出的代价。如果见热则压、遇冷则放，一味追求经济短期的

In [36]:
# Distribution of Final Sentiment
sentiment_distribution_final = annotation_final['Final_Sentiment'].value_counts()
sentiment_distribution_final = sentiment_distribution_final
print(sentiment_distribution_final)

# Distribution of Annotation State
sentiment_distribution_final = annotation_final['Annotation_State'].value_counts()
sentiment_distribution_final = sentiment_distribution_final
print(sentiment_distribution_final)

Final_Sentiment
 1    78830
 0    21169
-1        1
Name: count, dtype: int64
Annotation_State
consensus                    75581
difference_trust_ERNIE       14129
difference_trust_Senta       10143
ERNIE_missing_trust_Senta       74
Senta_missing_trust_ERNIE       72
both_missing                     1
Name: count, dtype: int64


In [37]:
final_sentiment_minus_1 = annotation_final[annotation_final['Final_Sentiment'] == -1]
final_sentiment_minus_1

,NewsID,Explicit_Company,News,Confidence_Senta,Final_Sentiment,Annotation_State
54602,614029,厦门国贸,海底捞2022年扭亏为盈 特海国际预亏逾3600万美元 日前，13个国家部委办局联合印发了《支持国有企业办医疗机构高质量发展工作方案》(以下简称《方案》)。 “国有企业办医疗机构是我国医疗卫生服务体系的重要组成部分。”上述《方案》提到。国企办医院，也将促进我国多层次多样化医疗服务的发展。 时针拨回到2021年12月25日。这一天，随着红布揭开，“厦门泰和康复医院”正式更名为“厦门国贸泰和康复医院”，标志着这家取得福建省第三张三级康复医院许可证的民营医院正式加入国有企业阵营。 “过往简单的、拿到执照和医保定点就赚钱的模式，必然被淘汰。”厦门国贸泰和康复医院副董事长吕锦松说道。 早在2020年，厦门国贸(SH600755，股价8.4元，市值185亿元)就已和医院创办人兼时任总经理的吕锦松接触。吕锦松认为，国资股东将在医改这场时代浪潮中，充分发挥资源整合作用，稳中奋进，改制创新。 挑战：盈利、社会责任及长期经营中如何找到平衡点？ 作为关乎老百姓切身利益的行业，医疗改革的每一步都是谨慎、小心的。“医疗的背后，是一条条生命，这是造‘福田’的公益事业。”吕锦松说道。 涉足医疗行业，与吕锦松的一次新加坡之行有关——在新加坡时，吕锦松深受“陈笃生医院”历史故事的撼动。这家成立近200年的医院，秉承创始人陈笃生“为新加坡人民增添健康寿命”的理想愿景，时至今日仍在为当地百姓的健康保驾护航。 深受触动的吕锦松请来美国护理科学院院士、新加坡国立大学护理学院副院长何红姑教授，作为名誉院长与他一同在鹭岛(地处福建厦门)创办了这家康复专科医院。 然而，在医院创立之初，厦门国贸泰和康复医院也曾面临固定资产投入、经营现金流短缺等难题，特别是2020年疫情的暴发，更让医院的发展雪上加霜。 “投了几千万，房子都抵押了，有没有想过这个事情实在不行就不做了？”被记者这么问起时，吕锦松只是说道：“其实只要整体发展趋势是在往上走的，每天进步一点，你看得到希望，心里就想也许还能再熬一熬。” 谈到加入“国字号”的原因，吕锦松表示，厦门国贸作为世界500强国贸控股集团的核心成员企业，肩负社会责任，并且对医院的运营模式与发展定位也较为认可。 在吕锦松看来，预防、治疗和康复这三个阶段都是医疗不可缺失的环节，这和厦门国贸将医疗服务业务定位为“公立医疗的有力补充”不谋而合。有了观念上的一致，国资股东与民营团队一拍即合，积极推进厦门国贸泰和康复医院的转型建设。 厦门国贸以供应链服务为主营业务，近年来开辟了健康科技的新兴赛道。对于医疗领域，厦门国贸并不是贸然进入，而是在原有医疗仪器设备供应链业务的积淀基础上，不断延伸拓展医疗器械产业链及健康养老服务，将健康科技新赛道纳入企业的“十四五”战略规划中。 “经过疫情三年，我们会发现目前依然存在医疗资源配置不均衡的情况。而并购康复医院，是为积极响应国家分级诊疗的政策，并向医疗产业链下游布局延伸。公立医院职能侧重治病救人，需要康复医院来承担使患者回归社会的职能，从而促进分级诊疗体系建设。”厦门国贸方面向《每日经济新闻》记者表示，“作为世界500强国有企业，同时在厦门本土立足四十年，与民营医疗机构结合显著提高了其公信力，不但获得了本地市民的更多认可，也促进(医院)吸引更多高端人才、专家主任。” “在新时代的要求下，国企办医也面临着更多新机遇、新挑战。如何在项目投资回收周期、国企社会责任及医院长期经营建设中找到平衡点，对国有企业来说，也是一个很大的课题。而通过多个健康细分赛道协同布局，有效发挥规模优势，或许可以实现边际效益的递增。”厦门国贸称。 上市公司、国企成增量来源 2月22日，华润医疗(HK01515，股价6.32港元，市值82亿港元)发布公告称，其全资附属公司华润医院投资(中国)有限公司(以下简称华润医院投资)与辽润管理有限公司(以下简称辽润管理)签订协议。 据上述协议，辽润管理有条件同意出售及华润医院投资有条件同意按对价人民币约34亿元购买辽宁华润健康76.10%股权和深圳华润健康100%股权。 值得注意的是，辽宁华润健康间接持有31家辽健成员机构的举办权，其中包括了辽宁省健康产业集团抚矿总医院、辽宁省健康产业集团抚矿脑科医院和辽宁省健康产业集团抚矿老虎台医院等医院或医疗机构。 针对上述交易，华润医疗表示：“本公司作为华润集团大健康板块的上市公司，持续贯彻落实健康中国建设战略部署和国务院国有资产监督管理委员会发布的《关于国有企业办教育医疗机构深化改革的指导意见》。” “本公司的业务规模将进一步扩大，无论是在所属医疗机构数量还是在合并收入规模角度。”华润医疗表示。 民营医院在我国的医疗市场当中处于数量多、规模小、诊疗人数少的状况。数据显示，截至2021年底，我国总计拥有医院合计约3.66万所，其中公立医院约1.18万所，民营医院约2.48万所。 虽然从数量上来看，民营医院是公立医院的两倍，但从诊疗人次上相较于公立医院却相距甚远。去年1月至9月，公立医院诊疗人次25亿人次，民营医院为4.8亿人次。 “民营医疗资本市场大浪淘沙，单体私人资本投资医院逐渐减少，无法实现高质量转型的莆田系医院出现倒逼、被收购现象，莆田系医院在民营医疗市场中的份额减少。与此同时，上市公司、国有企业不断进入医疗服务领域，成为民营医疗市场新增服务能力的主要来源。”《中国民营医院发展报告(2021)》中这么描述。 与此同时，近年来，国企医院改革取得了重要进展。在这轮大潮中，新里程健康集团、华润医疗等医疗控股集团迅速成长。 此外，在疫情等意外因素的冲击下，民营医院的抗风险能力较弱。《中国卫生健康统计年鉴》数据显示，2020年和2021年，非公立医疗卫生机构收入减支出分别约为-1307亿元和1211亿元，相较于公立医院的状况波动较大。 《方案》中也提到：“统筹规划、合理利用资源，充分发挥国有企业办医疗机构的优势，切实将符合条件的国有企业办医疗机构纳入各地院前急救网络、应急医疗救治体系和重大疫情救治体系。” 显然，中国的医疗体系在经历了疫情大考之后，更具有韧性。国企办医院，也将从多层次角度为我国医疗体系做出贡献。（文章来源：每日经济新闻） 文章来源：每日经济新闻 责任编辑：118,-1.0,-1,both_missing


In [38]:
only_missing_NewsID = final_sentiment_minus_1['NewsID'].values[0]

In [39]:
print(annotation_with_ERNIE[annotation_with_ERNIE['NewsID'] == only_missing_NewsID]['Sentiment'])

54602    {"id":"as-1pkghmcdwc","object":"chat.completion","created":1699183130,"result":"您所提到的问题，我没有找到相关的资料和数据，您可以试试我的其他能力。\r\n","is_truncated":false,"need_clear_history":true,"ban_round":-1,"usage":{"prompt_tokens":2257,"completion_tokens":31,"total_tokens":2288}}
Name: Sentiment, dtype: object


In [40]:
print(annotation_with_Senta[annotation_with_Senta['NewsID'] == only_missing_NewsID]['Sentiment'])

54602    {'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
Name: Sentiment, dtype: object


In [41]:
only_missing_News = annotation_with_Senta[annotation_with_Senta['NewsID'] == only_missing_NewsID]['News'].values[0]
print(only_missing_News)

海底捞2022年扭亏为盈 特海国际预亏逾3600万美元 日前，13个国家部委办局联合印发了《支持国有企业办医疗机构高质量发展工作方案》(以下简称《方案》)。 　　“国有企业办医疗机构是我国医疗卫生服务体系的重要组成部分。”上述《方案》提到。国企办医院，也将促进我国多层次多样化医疗服务的发展。 　　时针拨回到2021年12月25日。这一天，随着红布揭开，“厦门泰和康复医院”正式更名为“厦门国贸泰和康复医院”，标志着这家取得福建省第三张三级康复医院许可证的民营医院正式加入国有企业阵营。 　　“过往简单的、拿到执照和医保定点就赚钱的模式，必然被淘汰。”厦门国贸泰和康复医院副董事长吕锦松说道。 　　早在2020年，厦门国贸(SH600755，股价8.4元，市值185亿元)就已和医院创办人兼时任总经理的吕锦松接触。吕锦松认为，国资股东将在医改这场时代浪潮中，充分发挥资源整合作用，稳中奋进，改制创新。　　挑战：盈利、社会责任及长期经营中如何找到平衡点？  　　作为关乎老百姓切身利益的行业，医疗改革的每一步都是谨慎、小心的。“医疗的背后，是一条条生命，这是造‘福田’的公益事业。”吕锦松说道。 　　涉足医疗行业，与吕锦松的一次新加坡之行有关——在新加坡时，吕锦松深受“陈笃生医院”历史故事的撼动。这家成立近200年的医院，秉承创始人陈笃生“为新加坡人民增添健康寿命”的理想愿景，时至今日仍在为当地百姓的健康保驾护航。 　　深受触动的吕锦松请来美国护理科学院院士、新加坡国立大学护理学院副院长何红姑教授，作为名誉院长与他一同在鹭岛(地处福建厦门)创办了这家康复专科医院。 　　然而，在医院创立之初，厦门国贸泰和康复医院也曾面临固定资产投入、经营现金流短缺等难题，特别是2020年疫情的暴发，更让医院的发展雪上加霜。 　　“投了几千万，房子都抵押了，有没有想过这个事情实在不行就不做了？”被记者这么问起时，吕锦松只是说道：“其实只要整体发展趋势是在往上走的，每天进步一点，你看得到希望，心里就想也许还能再熬一熬。” 　　谈到加入“国字号”的原因，吕锦松表示，厦门国贸作为世界500强国贸控股集团的核心成员企业，肩负社会责任，并且对医院的运营模式与发展定位也较为认可。 　　在吕锦松看来，预防、治疗和康复这三个阶段都是医疗不可缺失的环节，这和厦门国贸将医疗服务业务定位为“公立医疗的有力补充”不谋而合。有了

Ask ChatGPT:"所以这条新闻对厦门国贸来说是正面的吗？"

ChatGPT Ans:
"从这条新闻来看，厦门国贸泰和康复医院的转型加入国有企业阵营被描述为一个积极的举措。文章提及了医疗行业面临的挑战，但也指出了企业团队积极推进医院的转型建设，通过多个健康细分赛道协同布局，发挥规模优势的意图。

这样的转型可能对厦门国贸是一个积极的发展。加入国有企业阵营可能带来更多的资源整合和支持，有助于医院的长期发展和提升社会责任感。然而，这也需要在盈利和社会责任之间找到平衡，所以成功与否还需时间来验证。"

In [42]:
annotation_final.loc[annotation_final['Final_Sentiment'] == -1, 'Annotation_State'] = "manually_ask_ChatGPT"
annotation_final.loc[annotation_final['Final_Sentiment'] == -1, 'Confidence_Senta'] = -1
annotation_final.loc[annotation_final['Final_Sentiment'] == -1, 'Final_Sentiment'] = 1

In [43]:
# Distribution of Final Sentiment
sentiment_distribution_final = annotation_final['Final_Sentiment'].value_counts()
sentiment_distribution_final = sentiment_distribution_final
print(sentiment_distribution_final)

# Distribution of Annotation State
sentiment_distribution_final = annotation_final['Annotation_State'].value_counts()
sentiment_distribution_final = sentiment_distribution_final
print(sentiment_distribution_final)

Final_Sentiment
1    78831
0    21169
Name: count, dtype: int64
Annotation_State
consensus                    75581
difference_trust_ERNIE       14129
difference_trust_Senta       10143
ERNIE_missing_trust_Senta       74
Senta_missing_trust_ERNIE       72
manually_ask_ChatGPT             1
Name: count, dtype: int64


In [44]:
annotation_final.to_excel(f'Training_dataset\\Training_set_with_final_annotation.xlsx', index=False)